In [3]:
!pip install transformers
!pip install tensorflow-gpu
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import numpy as np

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "/usr/local/lib/python3.11/dist-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/usr/local/lib/python3.11/dist-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/usr/local/lib/python3.11/dist-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        

In [7]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

eos_token = tokenizer.sep_token_id
pad_token = tokenizer.pad_token_id

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [3]:
input_sequences = []
target_tokens = []
max_length = 64

with open("dataset.txt", "r", encoding="utf-8") as file:
    for line in file:
        question, answer = line.strip().split('\t')

        # Encode the texts. Adding the EOS token is important for GPT to know when a sentence ends
        question_tokens = tokenizer.encode(question, add_special_tokens=False)
        answer_tokens = tokenizer.encode(answer, add_special_tokens=False) + [eos_token] # Ensure EOS token is added to the answer

        # Construct sequences of max_length or shorter
        for i in range(0, len(answer_tokens)):
            # Input sequence: question + end token + part of the answer
            seq = question_tokens + [eos_token] + answer_tokens[:i]
            if len(seq) > max_length:
                break

            # Ensure sequences are padded to have uniform length
            input_sequences.append(seq + [pad_token] * (max_length - len(seq)))  # Padding

            # Target token is the next token in the answer
            target_tokens.append(answer_tokens[i])

In [4]:
for seq in input_sequences[:10]:
    #print(seq)
    print(tokenizer.decode(np.array(seq), clean_up_tokenization_spaces=True))
    print('\n')
          
for seq in target_tokens[:10]:
    print(tokenizer.decode(np.array(seq), clean_up_tokenization_spaces=True))

hi, how are you doing? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


hi, how are you doing? [SEP] i [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


hi, how are you doing? [SEP] i'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [5]:
input_sequences = tf.constant(input_sequences, dtype=tf.int32)
target_tokens = tf.constant(target_tokens, dtype=tf.int32)
dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_tokens))
dataset = dataset.shuffle(buffer_size=1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

vocab_size = tokenizer.vocab_size

In [6]:
# Create the model

In [7]:
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
outputs = bert_model(input_ids)[0]  # We use only the last hidden state
cls_output = outputs[:, -1, :]  # We use the output of the last token for prediction
predictions = tf.keras.layers.Dense(vocab_size, activation='softmax')(cls_output)

model = tf.keras.Model(inputs=input_ids, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 64)]              0         
                                                                 
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPo   109482240 
 )                           olingAndCrossAttentions             
                             (last_hidden_state=(Non             
                             e, 64, 768),                        
                              pooler_output=(None, 7             
                             68),                                
                              past_key_values=None,              
                             hidden_states=None, att             
                             entions=None, cross_att             
                             entions=None)                       
                                                             

In [8]:
if tf.test.gpu_device_name():
    print(f"Default GPU Device: {tf.test.gpu_device_name()}")
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


2024-04-27 02:19:39.649993: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-27 02:19:39.650083: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-27 02:19:39.650103: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-27 02:19:39.650397: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-27 02:19:39.650410: I tensorflow/core/common_runtime/gpu/gpu

In [9]:
epochs = 15

# Train the model
history = model.fit(dataset, epochs=epochs)

Epoch 1/15


I0000 00:00:1714184416.702579   17828 service.cc:145] XLA service 0x7fc82371f0a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714184416.702627   17828 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-04-27 02:20:16.710284: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-27 02:20:16.729452: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1714184416.780708   17828 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1139/1139 [==============================] - 215s 150ms/step - loss: 5.6840
Epoch 2/15
1139/1139 [==============================] - 169s 148ms/step - loss: 5.2431
Epoch 3/15
1139/1139 [==============================] - 167s 147ms/step - loss: 5.1652
Epoch 4/15
1139/1139 [==============================] - 169s 148ms/step - loss: 5.1510
Epoch 5/15
1139/1139 [==============================] - 168s 147ms/step - loss: 5.1410
Epoch 6/15
1139/1139 [==============================] - 167s 147ms/step - loss: 5.1352
Epoch 7/15
1139/1139 [==============================] - 167s 147ms/step - loss: 5.1344
Epoch 8/15
1139/1139 [==============================] - 167s 147ms/step - loss: 5.1291
Epoch 9/15
1139/1139 [==============================] - 166s 146ms/step - loss: 5.1215
Epoch 10/15
1139/1139 [==============================] - 166s 146ms/step - loss: 5.1132
Epoch 11/15
1139/1139 [==============================] - 166s 146ms/step - loss: 5.1126
Epoch 12/15
1139/1139 [=============================

In [14]:
model_save_path = "my_transformer_model"
tf.saved_model.save(model, model_save_path)

KeyboardInterrupt: 

In [8]:
from tensorflow.keras.models import load_model

model_save_path = "my_transformer_model"
loaded_model = tf.saved_model.load(model_save_path)

In [9]:
def generate_until_sep_token(starting_text, max_tokens=50):
    text = starting_text
    for _ in range(max_tokens):
        token_ids = tokenizer.encode(text, add_special_tokens=False) + [eos_token]
        
        # Pad token_ids to ensure the input length is exactly 64
        if len(token_ids) < 64:
            token_ids += [pad_token] * (64 - len(token_ids))
        elif len(token_ids) > 64:
            token_ids = token_ids[-64:]  # Truncate to the last 64 tokens if exceeding the length
        
        input_sequence = tf.constant([token_ids], dtype=tf.int32)

        # Using the serving function with named arguments as expected by the model
        serving_fn = loaded_model.signatures['serving_default']
        predictions = serving_fn(input_ids=input_sequence)

        # Accessing the prediction using the 'dense' key
        predicted_token_id = tf.argmax(predictions['dense'], axis=-1).numpy()[0]

        if predicted_token_id == eos_token:
            break
        
        # Decode the predicted token ID to the corresponding token
        predicted_token = tokenizer.decode([predicted_token_id])
        text += predicted_token

    return text


In [10]:
# Now, you can test this function with your starting text
starting_text = "hello, how are you?"
generated_text = generate_until_sep_token(starting_text, max_tokens=50)
print("Generated text:", generated_text)

Generated text: hello, how are you?
